# 3.1 정확도 (Accyracy)
- 정확도: 실제 데이터에서 예측 데이터가 얼마나 같은지를 판단하는 지표
- Accuracy = (예측 결과가 동일 한 데이터 건수) / (전체 예측 데이터 건수)
	​


## 주의점
- 데이터의 구성에 따라 ML 모델의 성능을 왜곡할 수 있다
  - 정확도 수치 하나만으로 성능 평가하면 안 된다
  - 여러 가지 지표와 함께 적용해야 한다

e.g: 불균형한 레이블 데이터 세트 

In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self,X,y):
        pass
    
    # 입력값으로 들어오는 X 데이터 셋의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self,X):
        return np.zeros( (len(X), 1) , dtype=bool)

# 사이킷런의 내장 데이터 셋인 load_digits( )를 이용하여 MNIST 데이터 로딩
digits = load_digits()
digits.target == 7
# digits번호가 7번이면 True이고 이를 astype(int)로 1로 변환, 7번이 아니면 False이고 0으로 변환. 
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split( digits.data, y, random_state=11)

# 불균형한 레이블 데이터 분포도 확인. 
print('레이블 테스트 세트 크기 :', y_test.shape)
print('테스트 세트 레이블 0 과 1의 분포도')
print(pd.Series(y_test).value_counts())

# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train , y_train)
fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 하여도 정확도는:{:.3f}'.format(accuracy_score(y_test , fakepred)))

레이블 테스트 세트 크기 : (450,)
테스트 세트 레이블 0 과 1의 분포도
0    405
1     45
Name: count, dtype: int64
모든 예측을 0으로 하여도 정확도는:0.900


## 결론

정확도가 높은 이유는 클래스 불균형(대부분 샘플이 0) 때문이라서이다. 전부 0으로 찍어도 약 90%가 나오지만 이는 전혀 유용하지 않다(숫자 7을 전부 놓친다). 따라서 불균형 데이터에서는 정확도만 보지 말고 Confusion Matrix, Precision, Recall, F1, PR-AUC 등을 함께 확인해야 한다.

# 3.2 오차 형렬 (Confusion Matrix)

학습된 분류 모젤이 예측을 수행하면서 얼마나 헷갈리고 있는지 보여주는 지표

|                       | **예측 Negative(0)** | **예측 Posivtive(1)** |
|-----------------------|----------------------|------------------|
| **실제 Negative(0)**  | TN                   | FP               |
| **실제 Positive(1)**  | FN                   | TP               |


- TN: 예측값을 Negative 값 0으로 예측했고 실제 값은 Negative 값 0
- FP: 예측값을 Positive 값 1으로 예측했고 실제 값은 Negative 값 0
- FN: 예측값을 Negative 값 0으로 예측했고 실제 값은 Positive 값 1
- TP: 예측값을 Positve 값 0으로 예측했고 실제 값은 Positive 값 0.

In [2]:
from sklearn.metrics import confusion_matrix

# 앞절의 예측 결과인 fakepred와 실제 결과인 y_test의 Confusion Matrix출력
confusion_matrix(y_test , fakepred)

array([[405,   0],
       [ 45,   0]])



|                 | **예측 Negative (0)**                          | **예측 Positive (1)**                         |
|-----------------|-------------------------------------------------|-----------------------------------------------|
| **실제 Negative(7이 아님)** | **TN = 405** <br> (예측: 7 아님, 실제: 7 아님) | **FP = 0**  <br> (예측: 7, 실제: 7 아님)        |
| **실제 Positive(Digit 7)**  | **FN = 45**  <br> (예측: 7 아님, 실제: 7)     | **TP = 0**  <br> (예측: 7, 실제: 7)           |



정확도 오차 형렬상에서 다음과 같이 재정의될 수 있다:

**정확도 = 예측 결과와 실제 값이 동일한 건수/전체 데이터 수 = (TN + TP) / (TN + FP + FN + TP)**

# 3.3 정밀도와 재현율
정밀도와 재현율: Positive 데이터 세트의 예측 성능에 좀더 초점을 맞춘 평가 지표

- **정밀도 = TP / (FP + TP)**
- **재현율 = TP / (FN + TP)**

**1. 정밀도(Precision)**: 양성이라고 예측한 것 중 정말 양성인 비율.

예) 스팸 필터: 정상 메일을 스팸으로 막으면 곤란하므로 정밀도가 중요(위양성↓)

**2. 재현율(Recall)**: 실제 양성 중에서 맞게 잡아낸 비율

예) 질병 검진: 환자를 놓치면 위험하므로 재현율이 중요(위음성↓)



## 1) 정밀도/재현율 트레이드오프

- 임계값(Threshold) 조정으로 업무 특성에 맞게 정밀도 또는 재현율을 의도적으로 끌어올릴 수 있다.
- 두 지표는 트레이드오프 관계이므로 한쪽을 높이면 다른 쪽이 떨어지기 쉽다 → 목표에 맞는 균형/임계값 최적화가 필요하다.

- predict_proba(): 각 샘플이 클래스 0/1일 확률을 반환한다. 이진 분류면 [:,0]은 0의 확률, [:,1]은 1의 확률이다.

- Binarizer: 확률 + 임계값으로 0/1 라벨로 바꿔준다. 임계값을 낮추면 재현율이 오르고 정밀도는 내려가는 경향이 있다.

In [3]:
proba = [0.12, 0.38, 0.51, 0.72, 0.29]  # predict_proba()[:,1] 결과(양성 확률)

# 임계값 0.5
# 0.12→0, 0.38→0, 0.51→1, 0.72→1, 0.29→0  => [0, 0, 1, 1, 0]

# 임계값 0.3 (더 느슨)
# 0.12→0, 0.38→1, 0.51→1, 0.72→1, 0.29→0  => [0, 1, 1, 1, 0]


## 2) 정밀도/재현율의 맹점

- 정밀도가 100%가 되는 방법: 확실한 기준이 되는 경우만 Positive로 예측하고 나머지는 모두 Negative로 예측한다
- 재현율이 100%가 되는 방법: 모든 환자를 Positive로 예측하면 된다

# 3.4 F1 스코어
F1 스코어: 정밀도와 재현율을 결합한 지표

$F_1 = \dfrac{2}{\tfrac{1}{\text{recall}}+\tfrac{1}{\text{precision}}}
= 2\cdot\dfrac{\text{precision}\cdot\text{recall}}{\text{precision}+\text{recall}}$


# 3.5 ROC 곡선과 AUC
ROC 곡선과 이에 기반한 AUC 스코어는 이진 분류의 예측 성능 측정에서 중요하게 사용되는 지표다

- ROC 곡선은 FPR(False Positive Rate)이 변할 때 TPR(True Positive Rate)이 어떻게 변하는지를 나타내는 곡선이다

- 민감도(TPR)는 실제값 Positive(양성)가 정확히 예측돼야 하는 수준을 나타낸다 (질병이 있는 사람은 질병이 있는 것으 로양성판정)
- 특이성재(TNR)은 실제값 Negative(음성)가 정확히 예측돼야 하는 수준을 나타낸다 (질병이 없는 건강한 사람은 질병이 없는 것으로 음성 판정)

**FPR = FP / (FP + TN) = 1 - TNR = 1 - 특이성**